In [95]:
import pandas as pd
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from xbrl import XBRLParser
import io

In [13]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [4]:
link = (
    #"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
)
df = pd.read_html(link, header=0)[0]

In [21]:
str_df = df.astype(str)
str_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
497,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [110]:
report_url = "https://www.sec.gov/Archives/edgar/data/936468/0000936468-20-000016.txt"
report_response = requests.get(report_url, headers=get_random_headers())

soup = BeautifulSoup(report_response.text, 'html.parser')

xbrl_content = soup.find('xbrl')  # Adjust this to the correct tag or method to extract XBRL


In [ ]:
if xbrl_content:
    
    xbrl_data = str(xbrl_content)  # Keep data as string, not bytes
    xbrl_file_like = io.StringIO(xbrl_data)  # Use StringIO for string data
    xbrl_file_like.seek(0)
    

    # Parse the XBRL data using the file-like object
    try:
        xbrl_parser = XBRLParser()
        xbrl = xbrl_parser.parse(xbrl_file_like)
        print(xbrl)
    except Exception as e:
        print("Error parsing XBRL:", e)
else:
    print("XBRL content not found.")

TypeError: 'NoneType' object is not callable

In [78]:
def get_cik():
    link = (
        #"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"
        "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    )
    df = pd.read_html(link, header=0)[0]
    snp_str_df = df.astype(str)
    #print(type(str_ciks[0]))
    #cik_set = set(str_df["CIK"])
    #print(snp_str_df)
    snp_str_df['Count'] = snp_str_df.groupby('CIK').cumcount()

    snp_str_df = snp_str_df[snp_str_df['Count'] == 0]

    snp_str_df = snp_str_df.drop(columns=['Count'])

    return snp_str_df

In [79]:
df = get_cik()

In [80]:
set_df = set(df['CIK'])
print(len(df))


if '1564708' in set_df:
    print(True)

ticker = df.loc[df["CIK"] == '1564708', "Symbol"]

print(ticker)

499
True
337    NWSA
Name: Symbol, dtype: object


In [32]:
df[336: 340]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
336,NEM,Newmont,Materials,Gold,"Denver, Colorado",1969-06-30,1164727,1921
337,NWSA,News Corp (Class A),Communication Services,Publishing,"New York City, New York",2013-08-01,1564708,2013 (News Corporation 1980)
338,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708,2013 (News Corporation 1980)
339,NEE,NextEra Energy,Utilities,Multi-Utilities,"Juno Beach, Florida",1976-06-30,753308,1984 (1925)


In [73]:
def non_unique_cik(snp_str_df):
    res = snp_str_df["CIK"].value_counts()
    non_unique_ciks = res[res > 1]

    return non_unique_ciks.index


In [75]:
set(non_unique_cik(df))


{'1564708', '1652044', '1754301'}